# Intro to Built-In Functions from `contrib.functions`


## Initial Setup

Lets first import the necessary modules and define the agents.

In [1]:
import os
from autogen import AssistantAgent, UserProxyAgent
from autogen.agentchat.contrib.functions import youtube_utils as yt
from autogen.agentchat.contrib.functions import file_utils as fu

In [2]:
config_list = [
    {
        "model": "gpt-4",
        "api_key": os.environ.get("OPENAI_API_KEY"),
    }
]

assistant = AssistantAgent(name="coder", llm_config={"config_list": config_list, "cache": None})
user = UserProxyAgent(
    name="user",
    code_execution_config={
        "work_dir": "/tmp",
    },
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and "TERMINATE" in x.get("content", ""),
)

## Simple Example

In [3]:
assistant.register_for_llm(description="Fetch transcript of a youtube video")(yt.get_youtube_transcript)
user.register_for_execution()(yt.get_youtube_transcript)

result = user.initiate_chat(
    assistant,
    message="Please summarize the video: https://www.youtube.com/watch?v=9iqn1HhFJ6c",
    summary_method="last_msg",
)

user (to coder):

Please summarize the video: https://www.youtube.com/watch?v=9iqn1HhFJ6c

--------------------------------------------------------------------------------
coder (to user):

***** Suggested tool Call (call_FPOzM4IjGuiNKKlZhIKSTKRD): get_youtube_transcript *****
Arguments: 
{
"youtube_link": "https://www.youtube.com/watch?v=9iqn1HhFJ6c"
}
***************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_youtube_transcript...
requested package: youtube_transcript_api 0.6.0
found package youtube-transcript-api 0.6.0
user (to coder):

user (to coder):

***** Response from calling tool "call_FPOzM4IjGuiNKKlZhIKSTKRD" *****
now ai is a great thing because AI will solve all the problems that we have today it will solve employment it will solve disease it will solve poverty but it will also create new problems the problem of fake news is going

## Adavanced: Registering Multiple Functions

Lets import multiple functions and use them accomplish more complex tasks.

In [4]:
# register multiple file reading functions
for foo in [
    fu.read_text_from_image,
    fu.read_text_from_pdf,
    fu.read_text_from_docx,
    fu.read_text_from_pptx,
    fu.read_text_from_xlsx,
    fu.read_text_from_audio,
]:
    foo_desc = foo.__doc__  # get doctring of the function
    assistant.register_for_llm(description=foo_desc)(foo)
    user.register_for_execution()(foo)

In [5]:
dummy_png = "https://upload.wikimedia.org/wikipedia/commons/thumb/0/0f/Captioned_image_dataset_examples.jpg/1024px-Captioned_image_dataset_examples.jpg"
dummy_pdf = "https://www.w3.org/WAI/ER/tests/xhtml/testfiles/resources/pdf/dummy.pdf"
dummy_mp3 = "https://github.com/realpython/python-speech-recognition/raw/master/audio_files/harvard.wav"

result = user.initiate_chat(
    assistant,
    message=f"Please summarize the contents of the following files: {' '.join([dummy_png, dummy_pdf, dummy_mp3])}",
    summary_method="last_msg",
)

user (to coder):

Please summarize the contents of the following files: https://upload.wikimedia.org/wikipedia/commons/thumb/0/0f/Captioned_image_dataset_examples.jpg/1024px-Captioned_image_dataset_examples.jpg https://www.w3.org/WAI/ER/tests/xhtml/testfiles/resources/pdf/dummy.pdf https://github.com/realpython/python-speech-recognition/raw/master/audio_files/harvard.wav

--------------------------------------------------------------------------------
coder (to user):

***** Suggested tool Call (call_a62kEIHFUhyooITpart1yHpt): read_text_from_image *****
Arguments: 
{
  "file_path": "https://upload.wikimedia.org/wikipedia/commons/thumb/0/0f/Captioned_image_dataset_examples.jpg/1024px-Captioned_image_dataset_examples.jpg"
}
*************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION read_text_from_image...
requested package: pillow None
found package pi

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


user (to coder):

user (to coder):

***** Response from calling tool "call_a62kEIHFUhyooITpart1yHpt" *****
Error: cannot identify image file <_io.BytesIO object at 0x7fe465f9a610>
**********************************************************************

--------------------------------------------------------------------------------
coder (to user):

It seems like the system was not able to identify text from the provided image file. This could be due to complexities in the image like poor quality, complex background, or very stylized text. In some cases, images with captions may not be easily processed by optical character recognition (OCR) technology.

Let's process the next file which is a PDF document.
***** Suggested tool Call (call_pQP1zVHddWY9noBUazdBWiNq): read_text_from_pdf *****
Arguments: 
{
  "file_path": "https://www.w3.org/WAI/ER/tests/xhtml/testfiles/resources/pdf/dummy.pdf"
}
***********************************************************************************

------------

## Adavanced: Functions that Require Secrets

In this example, we will use a function that expects a secret, e.g., an `OPENAI_API_KEY` for it work. One such example is the function that using GPT-4-vision to perform image understanding.

In [6]:
assistant.register_for_llm(description="Use gpt4 vision to understand an image")(fu.caption_image_using_gpt4v)
user.register_for_execution()(fu.caption_image_using_gpt4v)

result = user.initiate_chat(
    assistant,
    message=f"Please summarize the contents of the following image using gpt4v: {dummy_png}",
    summary_method="last_msg",
)

user (to coder):

Please summarize the contents of the following image using gpt4v: https://upload.wikimedia.org/wikipedia/commons/thumb/0/0f/Captioned_image_dataset_examples.jpg/1024px-Captioned_image_dataset_examples.jpg

--------------------------------------------------------------------------------
coder (to user):

***** Suggested tool Call (call_jxjODN5UmMifdmnk8P1KIqir): caption_image_using_gpt4v *****
Arguments: 
{
  "file_path_or_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/0/0f/Captioned_image_dataset_examples.jpg/1024px-Captioned_image_dataset_examples.jpg"
}
******************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION caption_image_using_gpt4v...
requested package: openai None
found package openai 1.11.1
user (to coder):

user (to coder):

***** Response from calling tool "call_jxjODN5UmMifdmnk8P1KIqir" *****
{"finish_r